# Collecting Project Data Via Youtube Data API

Here we will collect all the data from the youtube API and save it to a file in the project data directory.

## Imports and API info

Here we will import necessary libraries, get API key from environment variables and set up the youtube data API

In [ ]:
# Import necessary libraries

import pandas as pd
from googleapiclient.discovery import build
from IPython.display import JSON
from dotenv import load_dotenv
import os
import isodate

In [ ]:
# Load API key from .env file

load_dotenv()
apiKey = os.getenv('API_KEY')


In [ ]:
# Builds youtube api
def buildYoutubeAPI():

    api_service_name = "youtube"
    api_version = "v3"

    # Get credentials and create an API client
    youtube = build(
        api_service_name, api_version, developerKey=apiKey)

    return youtube

## Selection of the Youtube Channels to be Analyzed

Here we will manually list all the youtube channels that we want to collect the data for. To do so we will need both the channel name and the channel ID

To find a youtube channel ID, you should:
1. go to the channel's youtube page, 
2. right click anywhere
3. select the option view page source
4. Press ctrl+f
5. search for ?channel_id

OBS.: the channel name does not have to be the exact channel name. The important thing is to get the correct channel ID and we will be able to get the official channel name via the youtube API.

In [ ]:
# Dictionary of the youtube channels 
# from which we will retrieve the data.
# The key is the channel name and the 
# value is the ID

channelIDDict = {
    "gorgonoid": "UCLfCo17TCjx7qf-JMhQioLQ",
    "noticiasMaromba": "UCmK5h2-a4CquS4nIxDN6j7g",
    #'albert': "UCij0YPRA_vGwKwCyn_o9MLw"
}

## Creating Functions to Retrieve Data

Now we will create the necessary functions to retrieve the data we want from the youtube data API.

### Retrieving Channel Statistics

Here we will retrieve data related to the channel as a whole (such as subscriber count).

First we will create a function that transforms the dictionary that maps the channel name to the channel ID into a string that contains all the channel IDs separated by commas.

In [ ]:
# Transforms the channel id dictionary to
# a string of the channel ID's separated
# by commas. Example: "id1, id2, id3"
def channelIDDictToCommaSeparatedIDString(channelIDDict):

    channelIDString = ""
    firstIDAdded = False
    for channel in channelIDDict:
        
        if firstIDAdded is False:
            channelIDString += channelIDDict[channel]
        else:
            channelIDString +=  ", " + channelIDDict[channel]

        firstIDAdded = True
    
    return channelIDString

Now we will create a function that gets the statistics we want from each of the youtube channels.

In [ ]:
# Gets the statistics of all channels
# included in the channel dictionary
def getAllChannelStatistics(channelIDDict): 

    allChannelStatistics = []
    
    request = buildYoutubeAPI().channels().list(
        part="snippet,contentDetails,statistics",
        id=channelIDDictToCommaSeparatedIDString(channelIDDict)
    )

    response = request.execute()

    # Loops through all channels and
    # adds data to dataframe
    for item in response['items']:
        channelStatistics = {
            "channelName": item['snippet']['title'],
            'subscriberCount': item['statistics']['subscriberCount'],
            'viewCount': item['statistics']['viewCount'],
            'videoCount': item['statistics']['videoCount'],
            'uploadedVideosPlaylistID': item['contentDetails']['relatedPlaylists']['uploads']
        }

        allChannelStatistics.append(channelStatistics)
        df = pd.DataFrame(allChannelStatistics)
    
    return df

Now we can use this function to retrieve information from the youtube channels we entered in the dictionary. Below, we will run a test to see if the channel information is being retrieved correctly.

In [ ]:
channelStatistics = getAllChannelStatistics(channelIDDict)

channelStatistics

### Retrieving Video Statistics

Here we will retrieve the data related to individual videos uploaded by channels (such as view count).

#### Retrieving Video IDs for All Videos of All Channels

All of the videos that were uploaded by a youtube channel are stored in a playlist that is identified by the playlist ID that we retrieved in the last dataframe we generated.

Now we will create a function that enables us to get the IDs of the videos that were uploaded by a specific channel for a given playlist ID.

In [ ]:
# Gets the IDs of the videos uploaded by
# a specific channel. OBS.: the playlist
# ID is the ID of the playlist that contains 
# all the videos of a channel.
# Returns a list that contains all of the
# channels video IDs
def getUploadedVideoIDs(playlistID):

    videoIDs = []

    youtube = buildYoutubeAPI()

    next_page_token = None
    while True:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlistID,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        for item in response['items']:
            videoIDs.append(item['contentDetails']['videoId'])
    
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return videoIDs

Now we can use this function to retrieve the video IDs of the videos the channel has uploaded. Below, we will run a test to see if the video IDs are being retrieved correctly.

In [ ]:
videoIDs = getUploadedVideoIDs('UULfCo17TCjx7qf-JMhQioLQ')

print(videoIDs)

len(videoIDs)

Now we will create a function that iterates through all of the playlist IDs in the channel statistics dataframe, gets all of the video ID's contained in each playlist and appends them to a list.

In [ ]:
# Retrieves all the video IDs of playlists 
# contained in whose IDs are contained in 
# a column of a dataframe
def getAllVideoIDs(channelStatisticsDataframe):
    allVideoIDs = []

    for playlistID in channelStatisticsDataframe['uploadedVideosPlaylistID']:
        allVideoIDs.extend(getUploadedVideoIDs(playlistID))

    return allVideoIDs

Now let's test if all the video IDs are being retrieved correctly

In [ ]:
videoIDs = getAllVideoIDs(channelStatistics)

print(videoIDs)

len(videoIDs)

#### Retrieving Youtube Video Categories

Now that we are able to get all video IDs from all the uploaded videos of all the channels we initially set in our dictionary, we can get the individual statistics for each video. 

But first, let's find out what are the names of the possible category IDs a video could have so that we can include the video category in the video statistics.

OBS.: the categories are set by youtube.

In [ ]:
# Gets all the youtube defined video 
# categories and adds them to a  
# dictionary that maps the id of the  
# category to the category title
def getVideoCategoriesBrazil():
    request = buildYoutubeAPI().videoCategories().list(
        part="snippet",
        regionCode='BR'
    )
    response = request.execute()

    categories = {}
    for item in response.get('items', []):
        id = item['id']
        title = item['snippet']['title']
        categories[id] = title
    
    return categories

Now let's see if we are retrieving the categories correctly.

In [ ]:
categories = getVideoCategoriesBrazil()

print(categories)

#### Retrieving The Video Statistics

Now we will create a funciton that retrieves the video statistics of all the videos contained in the list of video IDs we collected from all our selected channels.

In [ ]:
# Gets all video statistics for the 
# videos that were passed as argument 
# in video ID parameter
def getAllVideoStatistics(videoIDs):

    allVideoStatistics = []

    currentVideoRequestList = []

    videoCategories = getVideoCategoriesBrazil()

    # Iterate through videoIDs 50 at a time
    for i in range(0, len(videoIDs), 50):
        # Get next slice of 50 video IDs
        currentVideoRequestList = videoIDs[i:i+50]

        request = buildYoutubeAPI().videos().list(
            part="snippet,contentDetails,statistics",
            id=currentVideoRequestList,
            maxResults=50
        )
        response = request.execute()

        for item in response['items']:
            durationISO = item['contentDetails'].get('duration', None)
            if durationISO:
                durationTimeDelta = isodate.parse_duration(durationISO)
                durationSeconds = durationTimeDelta.total_seconds()
                hours = int(durationSeconds // 3600)
                minutes = int((durationSeconds % 3600) // 60)
                seconds = int(durationSeconds % 60)
            else:
                hours = 'videoDurationHours unknown'
                minutes = 'videoDurationMinutes unknown'
                seconds = 'videoDurationSeconds unknown'

            videoCategoryID = item['snippet'].get('categoryId', None)

            videoCategory = ''
            if videoCategoryID:
                videoCategory = videoCategories[item['snippet'].get('categoryId', None)]
            else:
                videoCategory = 'videoCategoryID unknown'

            videoStatistics = {
                "videoPublishDatetime": item['snippet'].get('publishedAt', 'videoPublishDatetime unknown'),
                "channelID": item['snippet'].get('channelId', 'channelID unknown'),
                "videoTitle": item['snippet'].get('title', 'videoTitle unknown'),
                "videoDescription": item['snippet'].get('description', 'videoDescription unknown'),
                "channelTitle": item['snippet'].get('channelTitle', 'channelTitle unknown'),
                "videoTags": item['snippet'].get('tags', []),
                "videoCategory": videoCategory,
                "isLiveBroadcastContent": item['snippet'].get('liveBroadcastContent', 'isLiveBroadcastContent unknown'),
                "videoDurationHours": hours,
                "videoDurationMinutes": minutes,
                "videoDurationSeconds": seconds,
                "videoUploadStatus": item.get('status', {}).get('uploadStatus', 'videoUploadStatus unknown'),
                "isPublicStatsViewable": item.get('status', {}).get('publicStatsViewable', 'isPublicStatsViewable unknown'),
                "videoViewCount": item.get('statistics', {}).get('viewCount', 'videoViewCount unknown'),
                "videoLikeCount": item.get('statistics', {}).get('likeCount', 'videoLikeCount unknown'),
                "videoCommentCount": item.get('statistics', {}).get('commentCount', 'videoCommentCount unknown'),
                "videoRecordingLocation": item.get('recordingDetails', {}).get('location', 'videoRecordingLocation unknown'),
                "videoRecordingDatetime": item.get('recordingDetails', {}).get('recordingDate', 'videoRecordingDatetime unknown')
                
            }
            
            allVideoStatistics.append(videoStatistics)
            df = pd.DataFrame(allVideoStatistics)

    return df


Now lets test if the video statistics are being retrieved correctly.

In [ ]:
videoStatistics = getAllVideoStatistics(videoIDs)

videoStatistics

#### Cleaning Data

Here we will clean data if necessary.

##### Cleaning Channel Statistics

In [ ]:
channelStatistics.info()

As shown above, the columns that are tipically supposed to have an int type (i.e. the columns that have a name ending in count) but are listed as objects. Let's convert the values of these columns to int to be more easily analyzed.

In [ ]:
columnsToConvert = ['subscriberCount', 'viewCount', 'videoCount']

cleanedChannelStatistics = channelStatistics

for column in columnsToConvert:
    cleanedChannelStatistics[column] = pd.to_numeric(channelStatistics[column], errors='coerce')

cleanedChannelStatistics.info()


##### Cleaning Video Statistics

In [ ]:
videoStatistics.info()

As shown above, there are columns that are tipically supposed to have an int type (such as the columns that have a name ending in count) but are listed as objects. Let's convert the values of these columns to int to be more easily analyzed.

In [ ]:
columnsToConvert = ['videoViewCount', 'videoLikeCount', 'videoCommentCount']

cleanedVideoStatistics = videoStatistics

for column in columnsToConvert:
    cleanedVideoStatistics[column] = pd.to_numeric(videoStatistics[column], errors='coerce').astype('Int64')

cleanedVideoStatistics.info()


As seen above, there are some videos that have null values in the video comment and like count. Let's analyze these videos below.

In [ ]:
nullLikeCount = cleanedVideoStatistics[cleanedVideoStatistics['videoLikeCount'].isnull()]

nullLikeCount

If the video like count value of a video is NaN, that means that the likes are not displayed publicly. You can verify this yourself searching for a video that has a NaN like count and another that does't. The video that has a NaN like count in the dataframe will not display the like count on youtube (but will still let you like the video). 

The same logic applies to comment count of videos that have a dataframe value of NaN in the video comment count.

Since we are still able to run operations such as sum() on a column that contains numpy NaN's, we will not be dropping these rows (videos) as they will not hinder our data analysis.

## Saving Statistics Dataframes to Project Data Directory

Now let's save the dataframes that contain the video and channel statistics to the data folder of the project.

In [ ]:
cleanedVideoStatistics.to_pickle('data/videoStatistics.pkl')

cleanedChannelStatistics.to_pickle('data/channelStatistics.pkl')

OBS.: we are saving the data as .pkl because it maintains data types such as numpy int64 and numpy float64 while csv files convert it to a simpler data type (not a numpy data type)

Here is a snippet of code to save the files as csv in case it is needed:

In [ ]:
cleanedVideoStatistics.to_csv('data/videoStatistics.csv')

cleanedChannelStatistics.to_csv('data/channelStatistics.csv')